<style>
    .ul{
        line-height: 250%;
    }
</style>

<font size="4">
    <ul>
        <li> calculate age by birth_time </li>
        <li> check is any people move contact_loc </li>
    </ul>
</font>


In [1]:
# basic libraries
import os
import math

import numpy as np
import pandas as pd
import matplotlib.pylab as plt

In [2]:
# pyspark related
from pyspark import SparkContext
from pyspark import SparkConf

In [3]:
sc = SparkContext('local[*]', 'PySpark')

In [4]:
# spark sql related
from pyspark.sql import DataFrameWriter
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
sql = sqlContext.sql

In [5]:
sc.master

u'local[*]'

<h1> Read Profile </h1>

In [6]:
profile_path = os.path.abspath('../dataset/profile/**/*.csv')

In [7]:
profile_txtRDD = sc.textFile(profile_path)
profile_txtRDD.take(5)

[u'1,1871111837,F,D,181903,F,373933,3320438400,1217.1769900503969,528.7309192220683,Y,Y,N,N,3696969600',
 u'2,1208855837,M,C,68383,F,72631,3688502400,1717.8806412199986,3.3682296721720717,Y,N,Y,N,3696969600',
 u'3,2880263837,M,E,457653,F,93331,3692304000,1216.7107394786008,307.4304767295353,N,N,N,Y,3696969600',
 u'4,1334999837,M,L,74257,I,248503,3647980800,1414.86660231857,888.3791238333672,Y,N,N,Y,3696969600',
 u'5,2817191837,F,A,45583,I,895863,3622406400,1388.8557936778625,642.1109125464978,Y,Y,N,Y,3696969600']

In [8]:
profile_txtRDD.count()

100

In [9]:
profiletxtnum = _
print profiletxtnum

100


In [10]:
from pyspark.sql.types import *

In [11]:
# field of profile
profilefields = [
    StructField('customer_id', StringType(), False),
    StructField('birth_time', LongType(), True),
    StructField('gender', StringType(), True),
    StructField('contact_loc', StringType(), True),
    StructField('contact_code', StringType(), True),
    StructField('register_loc', StringType(), True),
    StructField('register_code', StringType(), True),
    StructField('start_time', LongType(), True),
    StructField('aum', DoubleType(), True),
    StructField('net_profit', DoubleType(), True),
    StructField('credit_card_flag', StringType(), True),
    StructField('loan_flag', StringType(), True),
    StructField('deposit_flag', StringType(), True),
    StructField('wealth_flag', StringType(), True),
    StructField('partition_time', LongType(), True)
]

In [14]:
# construct profile's schema
profileschema = StructType(profilefields)

In [15]:
def process_profile_line(l):
    p = l.split(',')
    # for LongType
    for i in [1, 7, 14]:
        p[i] = int(p[i])
    # for DoubleType
    for i in [8, 9]:
        p[i] = float(p[i])
    return p

In [16]:
profile_df = profile_txtRDD \
                .map(process_profile_line) \
                .toDF(schema=profileschema)

profile_df.show(5)

+-----------+----------+------+-----------+------------+------------+-------------+----------+------------------+------------------+----------------+---------+------------+-----------+--------------+
|customer_id|birth_time|gender|contact_loc|contact_code|register_loc|register_code|start_time|               aum|        net_profit|credit_card_flag|loan_flag|deposit_flag|wealth_flag|partition_time|
+-----------+----------+------+-----------+------------+------------+-------------+----------+------------------+------------------+----------------+---------+------------+-----------+--------------+
|          1|1871111837|     F|          D|      181903|           F|       373933|3320438400|1217.1769900503969| 528.7309192220683|               Y|        Y|           N|          N|    3696969600|
|          2|1208855837|     M|          C|       68383|           F|        72631|3688502400|1717.8806412199986|3.3682296721720717|               Y|        N|           Y|          N|    3696969600|


<h1> Construct The Map of Contact Location and Contact Code </h1>

In [44]:
clRDD = profile_df.select('contact_loc')

In [45]:
ccRDD = profile_df.select('contact_code')

In [46]:
clccRDD = profile_df.select('contact_loc', 'contact_code')

<h2> Calculate Numbers </h2>

In [29]:
# clnum = clRDD.distinct().count()
clnum = 11
print "total " + str(clnum) + " different contact location"

total 11 different contact location


In [31]:
# ccnum = ccRDD.distinct().count()
clnum = 24
print "total " + str(ccnum) + " different contact code"

total 24 different contact code


In [33]:
# clccnum = clccRDD.distinct().count()
clccnum = 25
print "total " + str(clccnum) + " different pair of (location, code)"

total 25 different pair of (location, code)


<h2> Breif Survey </h2>

In [47]:
# calculate total amount of every location
cal_clRDD = clRDD.rdd \
                .map(lambda c: (c.contact_loc, 1)) \
                .reduceByKey(lambda x, y: x+y) \
                .sortByKey()

In [48]:
calcldict = cal_clRDD.collectAsMap()

In [51]:
print calcldict

{u'A': 4, u'C': 8, u'B': 6, u'E': 8, u'D': 8, u'G': 12, u'F': 4, u'H': 10, u'K': 8, u'J': 16, u'L': 16}


<h2> Check whether code is 1-1 to loc </h2>

In [22]:
check_codeTolocRDD = clccRDD.rdd \
                        .map(lambda c: (c.contact_code, [c.contact_loc])) \
                        .reduceByKey(lambda x, y: x+y) \
                        .filter(lambda (x, y): len(y) > 1)

In [34]:
# check = check_codeTolocRDD.count()
check = 1

if check == 0:
    print "1-1 checked"
else:
    print "total " + str(check) + " code have at least two location"
    print check_codeTolocRDD.take(5)

total 1 code have at least two location
[(u'457653', [u'J', u'E'])]


<h2> Construct Map </h2>

In [37]:
maplocTocodeRDD = clccRDD.rdd \
                    .map(lambda c: (c.contact_loc, [c.contact_code])) \
                    .reduceByKey(lambda x, y: x+y)

In [39]:
maplocTocodedict = maplocTocodeRDD.collectAsMap()

In [43]:
for x in maplocTocodedict.keys():
    print x
    print maplocTocodedict[x]
    print "total " + str(len(maplocTocodedict[x])) + " different code\n"

A
[u'45583']
total 1 different code

C
[u'437583', u'68383']
total 2 different code

B
[u'698061', u'115261']
total 2 different code

E
[u'457653', u'468541']
total 2 different code

D
[u'181903']
total 1 different code

G
[u'230881', u'215761', u'180201']
total 3 different code

F
[u'647221']
total 1 different code

H
[u'530713', u'159601', u'922561']
total 3 different code

K
[u'843643', u'109893']
total 2 different code

J
[u'32581', u'457653', u'48181', u'124963']
total 4 different code

L
[u'354621', u'358203', u'53593', u'74257']
total 4 different code



In [42]:
# final check
tol = 0
for x in maplocTocodedict.keys():
    tol += len(maplocTocodedict[x])

if tol == clccnum:
    print "Final check complete!"
else:
    print "Something goes wrong! Check it again"

Final check complete!


<h1> Check Migration </h1>

In [52]:
idclRDD = profile_df.select('customer_id', 'contact_loc')